In [1]:
path = ''
import re
with open(path+'civil_english.txt') as w:
    l = w.read()
w.close()
def remove_content_within_brackets(test_str):
    test_str = re.sub(r'(?m)^\*.*\n?', '', test_str)
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret
#l=remove_content_within_brackets(l)
l = re.sub(r'Chapter\s\d\s.*\n|Chapter\s.*\n','',l)
l = re.sub(r'Section\s\d\s.*\n|Section\s.*\n','',l)
#l = re.sub(r'Section\s.*\n','',l)
#f = re.split(r'Subsection\s.*\nArticle|Section\s.+\nArticle|\(.+\)\nArticle|.*\)\nArticle|\.\nArticle',l)
f = re.split(r'\nArticle',l)
print(f[2])
print(f[3])
for i in range(len(f)):
    pattern = re.sub(r'Subsection\s.*\n|Section\s.*\n','',f[i])
    f[i] = pattern
f.remove(f[0])
data = {}
pattern = re.compile(r'\s\d+\s|\s\d+-\d+\s')
for i in f:
    try:
        key = pattern.findall(i)[0]
        data[key] = i
    except:
        continue
list_of_keys = list(data.keys())
articles = [data[list_of_keys[i]] for i in range(len(data))]


 2 This Code must be construed in accordance with honoring the
dignity of individuals and the essential equality of both sexes.
 3 (1) The enjoyment of private rights shall commence at birth.
(2) Unless otherwise provided by applicable laws, regulations or treaties,
foreign nationals shall enjoy private rights.
(Age of Majority)


In [23]:
filename = 'riteval_H20.xml'
from xml.etree import cElementTree as ET
path = ''
with open(path+filename) as r:
    xmlstr = r.read()
r.close()
root = ET.fromstring(xmlstr)
t2 = []
for page in list(root):
    t2.append(page.find('t2').text)#t2 contains queries(for testing) as a list
from xml.etree import cElementTree as ET


In [24]:
path = ''
with open(path+filename) as r:
    xmlstr = r.read()
r.close()
root = ET.fromstring(xmlstr)
t1 = []

for page in list(root):
    tmp= page.find('t1').text
    tmp = remove_content_within_brackets(tmp)
    t1.append(tmp[8:12])

In [25]:
from xml.dom import minidom
def getText(nodelist):
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE :
            return node.data.strip()

def conv2pd(filename):
    print(filename)
    xmldoc = minidom.parseString(xmlstr)
    itemlist = xmldoc.getElementsByTagName('pair')
    NUM_ELEM = len(itemlist)
    ids = []
    val = []
    for i in range(NUM_ELEM):
        ids.append(itemlist[i].attributes['id'].value)
        val.append(itemlist[i].attributes['label'].value)
    return ids,val

pair_names,val = conv2pd(filename)

riteval_H20.xml


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())
def filt(case_docs):
    filtered_words =[]
    filtered_docs = [ '' for i in range(len(case_docs))]
    lemma_docs = [ '' for i in range(len(case_docs))]
    for i in range(len(case_docs)):
        case_words = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", case_docs[i])
        filtered_word_list = [word.lower() for word in case_words if (( len(word) >= 3 and word.isalpha() and word.lower() not in stop_words ))  ]
        filtered_words.append(filtered_word_list)
        '''or ( len(word) <= 3 and word.isdigit())'''
        for word in filtered_word_list:
            filtered_docs[i] = filtered_docs[i] + word + " "
            lemma_docs[i] = lemma_docs[i] + lemma.lemmatize(word) + " "
    return lemma_docs
lst = t2 + articles
lst = filt(lst)
vect = TfidfVectorizer()
tfidf = vect.fit_transform(lst)
tf_idf = (tfidf * tfidf.T).A
arr = []
for elem in lst:
    arr.append(elem.split())
from gensim.summarization.bm25 import get_bm25_weights
corpus = arr
result = get_bm25_weights(corpus, n_jobs=-1)

In [31]:
#TF-IDF
def calc(idx):
    if val[idx] == 'Y':
        scores = []
        for i in range(len(lst)-len(t2)):
            scores.append((tf_idf[idx][i+len(t2)],list_of_keys[i]))
        scores.sort(reverse=True)
        x = scores[0][1].strip()
        for i in range(len(x)):
            if x[i] == "-":
                print(int(scores[0][1][:i+1].strip()),t1[idx])
                return
        print(int(scores[0][1].strip()),t1[idx])
        
for i in range(len(t2)):
    calc(i)

6  6A 
738  9
A
588  13A
876  15W
108  108
103  99A
301  296
298  298
345  350
446  446
447  447
460  460
494  494
494  495
558  555
587  588
608  608
623  623
653  648
305  306
680  679
676  676
714  714
715  715
715  715


In [28]:
len(list_of_keys)

1056

In [32]:
#BM25
def calc(idx):
    if val[idx] == 'Y':
        scores = []
        for i in range(len(lst)-len(t2)):
            scores.append((result[idx][i+len(t2)],list_of_keys[i]))
        scores.sort(reverse=True)
        x = scores[0][1].strip()
        for i in range(len(x)):
            if x[i] == "-":
                print(int(scores[0][1][:i+1].strip()),t1[idx])
                return
        print(int(scores[0][1].strip()),t1[idx])
        
for i in range(len(t2)):
    calc(i)

6  6A 
738  9
A
113  13A
876  15W
108  108
103  99A
301  296
298  298
345  350
446  446
447  447
460  460
494  494
494  495
566  555
587  588
608  608
623  623
653  648
303  306
680  679
676  676
713  714
715  715
715  715


In [22]:
#TF-IDF
def calc(idx):
    scores = []
    for i in range(len(lst)-len(t2)):
        scores.append((tf_idf[idx][i+len(t2)],list_of_keys[i]))
    scores.sort(reverse=True)
    for rank in range(100):
        x = scores[rank][1].strip()
        flag = True
        s = ""
        for i in range(len(x)):
            if x[i] == "-":
                break
            else:
                s += x[i]
        print(pair_names[idx],"Q0",int(s.strip()),rank+1,format(scores[rank][0],'0.6f'),"iitptfidf")
        
for i in range(len(t2)):
    calc(i)

H19-1-3 Q0 96 1 0.615907 iitptfidf
H19-1-3 Q0 98 2 0.488451 iitptfidf
H19-1-3 Q0 95 3 0.446312 iitptfidf
H19-1-3 Q0 93 4 0.432711 iitptfidf
H19-1-3 Q0 94 5 0.360596 iitptfidf
H19-1-3 Q0 97 6 0.359028 iitptfidf
H19-1-3 Q0 99 7 0.357988 iitptfidf
H19-1-3 Q0 540 8 0.329038 iitptfidf
H19-1-3 Q0 506 9 0.273850 iitptfidf
H19-1-3 Q0 466 10 0.258034 iitptfidf
H19-1-3 Q0 98 11 0.242341 iitptfidf
H19-1-3 Q0 549 12 0.238855 iitptfidf
H19-1-3 Q0 537 13 0.209287 iitptfidf
H19-1-3 Q0 526 14 0.202036 iitptfidf
H19-1-3 Q0 101 15 0.198214 iitptfidf
H19-1-3 Q0 100 16 0.194814 iitptfidf
H19-1-3 Q0 112 17 0.190306 iitptfidf
H19-1-3 Q0 120 18 0.188638 iitptfidf
H19-1-3 Q0 747 19 0.188592 iitptfidf
H19-1-3 Q0 505 20 0.177921 iitptfidf
H19-1-3 Q0 407 21 0.176466 iitptfidf
H19-1-3 Q0 90 22 0.176359 iitptfidf
H19-1-3 Q0 109 23 0.174244 iitptfidf
H19-1-3 Q0 176 24 0.172608 iitptfidf
H19-1-3 Q0 556 25 0.172374 iitptfidf
H19-1-3 Q0 123 26 0.162649 iitptfidf
H19-1-3 Q0 859 27 0.161388 iitptfidf
H19-1-3 Q0 542 28 0

H19-5-3 Q0 166 22 0.173149 iitptfidf
H19-5-3 Q0 195 23 0.169568 iitptfidf
H19-5-3 Q0 476 24 0.168624 iitptfidf
H19-5-3 Q0 163 25 0.165675 iitptfidf
H19-5-3 Q0 205 26 0.161862 iitptfidf
H19-5-3 Q0 161 27 0.156747 iitptfidf
H19-5-3 Q0 293 28 0.155612 iitptfidf
H19-5-3 Q0 352 29 0.153278 iitptfidf
H19-5-3 Q0 289 30 0.150874 iitptfidf
H19-5-3 Q0 201 31 0.146877 iitptfidf
H19-5-3 Q0 184 32 0.146149 iitptfidf
H19-5-3 Q0 252 33 0.146027 iitptfidf
H19-5-3 Q0 258 34 0.144687 iitptfidf
H19-5-3 Q0 478 35 0.143701 iitptfidf
H19-5-3 Q0 898 36 0.143679 iitptfidf
H19-5-3 Q0 263 37 0.143658 iitptfidf
H19-5-3 Q0 251 38 0.142790 iitptfidf
H19-5-3 Q0 477 39 0.139672 iitptfidf
H19-5-3 Q0 758 40 0.134359 iitptfidf
H19-5-3 Q0 916 41 0.127871 iitptfidf
H19-5-3 Q0 180 42 0.126646 iitptfidf
H19-5-3 Q0 369 43 0.125493 iitptfidf
H19-5-3 Q0 253 44 0.124907 iitptfidf
H19-5-3 Q0 112 45 0.121825 iitptfidf
H19-5-3 Q0 716 46 0.121795 iitptfidf
H19-5-3 Q0 255 47 0.117130 iitptfidf
H19-5-3 Q0 564 48 0.116781 iitptfidf
H

H19-6-O Q0 303 55 0.104812 iitptfidf
H19-6-O Q0 174 56 0.104474 iitptfidf
H19-6-O Q0 548 57 0.104053 iitptfidf
H19-6-O Q0 156 58 0.103254 iitptfidf
H19-6-O Q0 398 59 0.101998 iitptfidf
H19-6-O Q0 145 60 0.101816 iitptfidf
H19-6-O Q0 398 61 0.099633 iitptfidf
H19-6-O Q0 345 62 0.099483 iitptfidf
H19-6-O Q0 162 63 0.099213 iitptfidf
H19-6-O Q0 637 64 0.098689 iitptfidf
H19-6-O Q0 293 65 0.094680 iitptfidf
H19-6-O Q0 160 66 0.093268 iitptfidf
H19-6-O Q0 147 67 0.092852 iitptfidf
H19-6-O Q0 944 68 0.092611 iitptfidf
H19-6-O Q0 157 69 0.091442 iitptfidf
H19-6-O Q0 314 70 0.091265 iitptfidf
H19-6-O Q0 930 71 0.090410 iitptfidf
H19-6-O Q0 520 72 0.089240 iitptfidf
H19-6-O Q0 174 73 0.087736 iitptfidf
H19-6-O Q0 374 74 0.086976 iitptfidf
H19-6-O Q0 810 75 0.086585 iitptfidf
H19-6-O Q0 173 76 0.086449 iitptfidf
H19-6-O Q0 711 77 0.084207 iitptfidf
H19-6-O Q0 195 78 0.084013 iitptfidf
H19-6-O Q0 422 79 0.083069 iitptfidf
H19-6-O Q0 149 80 0.083064 iitptfidf
H19-6-O Q0 158 81 0.082847 iitptfidf
H

H19-10-O Q0 406 72 0.095033 iitptfidf
H19-10-O Q0 786 73 0.094350 iitptfidf
H19-10-O Q0 664 74 0.092235 iitptfidf
H19-10-O Q0 592 75 0.092225 iitptfidf
H19-10-O Q0 703 76 0.091160 iitptfidf
H19-10-O Q0 272 77 0.090534 iitptfidf
H19-10-O Q0 212 78 0.089649 iitptfidf
H19-10-O Q0 748 79 0.088841 iitptfidf
H19-10-O Q0 708 80 0.087901 iitptfidf
H19-10-O Q0 605 81 0.087291 iitptfidf
H19-10-O Q0 182 82 0.085213 iitptfidf
H19-10-O Q0 617 83 0.083903 iitptfidf
H19-10-O Q0 333 84 0.081069 iitptfidf
H19-10-O Q0 414 85 0.080941 iitptfidf
H19-10-O Q0 235 86 0.079357 iitptfidf
H19-10-O Q0 777 87 0.078754 iitptfidf
H19-10-O Q0 266 88 0.077368 iitptfidf
H19-10-O Q0 370 89 0.076962 iitptfidf
H19-10-O Q0 211 90 0.076779 iitptfidf
H19-10-O Q0 86 91 0.076616 iitptfidf
H19-10-O Q0 466 92 0.076023 iitptfidf
H19-10-O Q0 590 93 0.075591 iitptfidf
H19-10-O Q0 691 94 0.075474 iitptfidf
H19-10-O Q0 224 95 0.075123 iitptfidf
H19-10-O Q0 759 96 0.075071 iitptfidf
H19-10-O Q0 358 97 0.074980 iitptfidf
H19-10-O Q0 5

H19-12-4 Q0 745 72 0.078305 iitptfidf
H19-12-4 Q0 642 73 0.077279 iitptfidf
H19-12-4 Q0 465 74 0.077209 iitptfidf
H19-12-4 Q0 747 75 0.076651 iitptfidf
H19-12-4 Q0 703 76 0.075754 iitptfidf
H19-12-4 Q0 117 77 0.075141 iitptfidf
H19-12-4 Q0 589 78 0.074345 iitptfidf
H19-12-4 Q0 661 79 0.074126 iitptfidf
H19-12-4 Q0 172 80 0.072614 iitptfidf
H19-12-4 Q0 548 81 0.072546 iitptfidf
H19-12-4 Q0 173 82 0.072445 iitptfidf
H19-12-4 Q0 115 83 0.072093 iitptfidf
H19-12-4 Q0 398 84 0.071985 iitptfidf
H19-12-4 Q0 707 85 0.071975 iitptfidf
H19-12-4 Q0 537 86 0.070684 iitptfidf
H19-12-4 Q0 565 87 0.070635 iitptfidf
H19-12-4 Q0 273 88 0.070582 iitptfidf
H19-12-4 Q0 186 89 0.070556 iitptfidf
H19-12-4 Q0 930 90 0.070443 iitptfidf
H19-12-4 Q0 446 91 0.070184 iitptfidf
H19-12-4 Q0 607 92 0.070032 iitptfidf
H19-12-4 Q0 346 93 0.069963 iitptfidf
H19-12-4 Q0 770 94 0.069795 iitptfidf
H19-12-4 Q0 471 95 0.069440 iitptfidf
H19-12-4 Q0 768 96 0.068677 iitptfidf
H19-12-4 Q0 533 97 0.068003 iitptfidf
H19-12-4 Q0 

H19-13-E Q0 370 42 0.163712 iitptfidf
H19-13-E Q0 404 43 0.156848 iitptfidf
H19-13-E Q0 378 44 0.155213 iitptfidf
H19-13-E Q0 465 45 0.149014 iitptfidf
H19-13-E Q0 577 46 0.148376 iitptfidf
H19-13-E Q0 380 47 0.139753 iitptfidf
H19-13-E Q0 295 48 0.138803 iitptfidf
H19-13-E Q0 379 49 0.137664 iitptfidf
H19-13-E Q0 381 50 0.136704 iitptfidf
H19-13-E Q0 573 51 0.134224 iitptfidf
H19-13-E Q0 398 52 0.129456 iitptfidf
H19-13-E Q0 303 53 0.121086 iitptfidf
H19-13-E Q0 339 54 0.115584 iitptfidf
H19-13-E Q0 1036 55 0.115430 iitptfidf
H19-13-E Q0 465 56 0.114735 iitptfidf
H19-13-E Q0 574 57 0.112010 iitptfidf
H19-13-E Q0 702 58 0.109824 iitptfidf
H19-13-E Q0 465 59 0.109413 iitptfidf
H19-13-E Q0 552 60 0.109324 iitptfidf
H19-13-E Q0 575 61 0.102195 iitptfidf
H19-13-E Q0 426 62 0.101226 iitptfidf
H19-13-E Q0 360 63 0.100792 iitptfidf
H19-13-E Q0 610 64 0.100104 iitptfidf
H19-13-E Q0 402 65 0.098930 iitptfidf
H19-13-E Q0 172 66 0.097751 iitptfidf
H19-13-E Q0 491 67 0.097336 iitptfidf
H19-13-E Q0

H19-16-1 Q0 423 88 0.046416 iitptfidf
H19-16-1 Q0 655 89 0.046371 iitptfidf
H19-16-1 Q0 751 90 0.045964 iitptfidf
H19-16-1 Q0 808 91 0.045878 iitptfidf
H19-16-1 Q0 178 92 0.044990 iitptfidf
H19-16-1 Q0 567 93 0.044929 iitptfidf
H19-16-1 Q0 505 94 0.044731 iitptfidf
H19-16-1 Q0 502 95 0.043951 iitptfidf
H19-16-1 Q0 516 96 0.043717 iitptfidf
H19-16-1 Q0 398 97 0.043228 iitptfidf
H19-16-1 Q0 508 98 0.042735 iitptfidf
H19-16-1 Q0 631 99 0.042287 iitptfidf
H19-16-1 Q0 122 100 0.042239 iitptfidf
H19-16-2 Q0 387 1 0.670788 iitptfidf
H19-16-2 Q0 602 2 0.363062 iitptfidf
H19-16-2 Q0 621 3 0.309237 iitptfidf
H19-16-2 Q0 386 4 0.299696 iitptfidf
H19-16-2 Q0 617 5 0.294841 iitptfidf
H19-16-2 Q0 597 6 0.293997 iitptfidf
H19-16-2 Q0 619 7 0.278503 iitptfidf
H19-16-2 Q0 374 8 0.274543 iitptfidf
H19-16-2 Q0 312 9 0.230648 iitptfidf
H19-16-2 Q0 581 10 0.227391 iitptfidf
H19-16-2 Q0 605 11 0.213718 iitptfidf
H19-16-2 Q0 336 12 0.204832 iitptfidf
H19-16-2 Q0 378 13 0.197681 iitptfidf
H19-16-2 Q0 272 14 0

H19-17-1 Q0 721 5 0.219699 iitptfidf
H19-17-1 Q0 620 6 0.202574 iitptfidf
H19-17-1 Q0 421 7 0.192154 iitptfidf
H19-17-1 Q0 232 8 0.190037 iitptfidf
H19-17-1 Q0 455 9 0.189774 iitptfidf
H19-17-1 Q0 212 10 0.189646 iitptfidf
H19-17-1 Q0 599 11 0.184605 iitptfidf
H19-17-1 Q0 722 12 0.174188 iitptfidf
H19-17-1 Q0 420 13 0.165967 iitptfidf
H19-17-1 Q0 723 14 0.164818 iitptfidf
H19-17-1 Q0 541 15 0.158173 iitptfidf
H19-17-1 Q0 477 16 0.157236 iitptfidf
H19-17-1 Q0 447 17 0.155560 iitptfidf
H19-17-1 Q0 489 18 0.155150 iitptfidf
H19-17-1 Q0 487 19 0.153569 iitptfidf
H19-17-1 Q0 488 20 0.146919 iitptfidf
H19-17-1 Q0 248 21 0.146354 iitptfidf
H19-17-1 Q0 475 22 0.145395 iitptfidf
H19-17-1 Q0 493 23 0.143774 iitptfidf
H19-17-1 Q0 709 24 0.139520 iitptfidf
H19-17-1 Q0 720 25 0.135852 iitptfidf
H19-17-1 Q0 482 26 0.135101 iitptfidf
H19-17-1 Q0 705 27 0.134818 iitptfidf
H19-17-1 Q0 710 28 0.132595 iitptfidf
H19-17-1 Q0 566 29 0.130351 iitptfidf
H19-17-1 Q0 415 30 0.129857 iitptfidf
H19-17-1 Q0 631 3

H19-26-O Q0 112 5 0.263470 iitptfidf
H19-26-O Q0 466 6 0.212015 iitptfidf
H19-26-O Q0 94 7 0.209327 iitptfidf
H19-26-O Q0 515 8 0.191940 iitptfidf
H19-26-O Q0 110 9 0.190632 iitptfidf
H19-26-O Q0 364 10 0.188603 iitptfidf
H19-26-O Q0 689 11 0.185348 iitptfidf
H19-26-O Q0 178 12 0.184009 iitptfidf
H19-26-O Q0 467 13 0.170205 iitptfidf
H19-26-O Q0 628 14 0.166745 iitptfidf
H19-26-O Q0 537 15 0.162434 iitptfidf
H19-26-O Q0 761 16 0.159560 iitptfidf
H19-26-O Q0 854 17 0.158174 iitptfidf
H19-26-O Q0 654 18 0.157889 iitptfidf
H19-26-O Q0 653 19 0.157389 iitptfidf
H19-26-O Q0 96 20 0.155138 iitptfidf
H19-26-O Q0 945 21 0.149655 iitptfidf
H19-26-O Q0 111 22 0.145614 iitptfidf
H19-26-O Q0 631 23 0.144305 iitptfidf
H19-26-O Q0 555 24 0.143310 iitptfidf
H19-26-O Q0 98 25 0.142853 iitptfidf
H19-26-O Q0 759 26 0.142131 iitptfidf
H19-26-O Q0 505 27 0.139871 iitptfidf
H19-26-O Q0 533 28 0.139507 iitptfidf
H19-26-O Q0 549 29 0.138427 iitptfidf
H19-26-O Q0 657 30 0.137508 iitptfidf
H19-26-O Q0 756 31 0

In [ ]:
#TF-IDF
def calc(idx):
    scores = []
    for i in range(len(lst)-len(t2)):
        scores.append((result[idx][i+len(t2)],list_of_keys[i]))
    scores.sort(reverse=True)
    for rank in range(100):
        x = scores[rank][1].strip()
        flag = True
        s = ""
        for i in range(len(x)):
            if x[i] == "-":
                break
            else:
                s += x[i]
        print(pair_names[idx],"Q0",int(s.strip()),rank+1,format(scores[rank][0],'0.6f'),"iitpBM25")
        
for i in range(len(t2)):
    calc(i)